In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras_tuner import RandomSearch

last_path = [[0,0,0,0,0],[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1],[1,1,0,0,0],[1,0,1,0,0],[1,0,0,1,0],[1,0,0,0,1],
             [0,1,1,0,0],[0,1,0,1,0],[0,1,0,0,1],[0,0,1,1,0],[0,0,1,0,1],[0,0,0,1,1],[0,0,1,1,1],[0,1,0,1,1],[0,1,1,0,1],[0,1,1,1,0],
             [1,0,0,1,1],[1,0,1,0,1],[1,0,1,1,0],[1,1,0,0,1],[1,1,0,1,0],[1,1,1,0,0],[0,1,1,1,1],[1,0,1,1,1],[1,1,0,1,1],[1,1,1,0,1],
             [1,1,1,1,0],[1,1,1,1,1]]
prefix = [f"{i:03d}" for i in range(593, 625)]
prefix_to_label = dict(zip(prefix, last_path))

def process_dataset(root_folder):
    image_paths = []
    label_data = []

    for roots, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.jpg'):
                # 파일 이름 분석을 위해 숫자만 추출
                prefix = file[0:3]
                
                # 접두사에 따른 레이블 할당
                label = prefix_to_label.get(prefix)
                
                # 유효한 레이블이 있는 경우에만 리스트에 추가
                if label is not None:
                    image_paths.append(os.path.join(roots, file))
                    label_data.append(label)
    
    return image_paths, label_data

# 각각의 데이터셋에 대해 함수를 호출
train_folder = r'E:\AI\dataset_skeleton_sep\face\KneePushup\training'
valid_folder = r'E:\AI\dataset_skeleton_sep\face\KneePushup\validation'
test_folder = r'E:\AI\dataset_skeleton_sep\face\KneePushup\test'

train_image_paths, train_label_data = process_dataset(train_folder)
valid_image_paths, valid_label_data = process_dataset(valid_folder)
test_image_paths, test_label_data = process_dataset(test_folder)

# 필요에 따라 결과를 확인하거나 다른 처리를 수행
print(len(train_image_paths), len(train_label_data))
print(len(valid_image_paths), len(valid_label_data))
print(len(test_image_paths), len(test_label_data))


31146 31146
6689 6689
6676 6676


In [2]:
# ImageDataGenerator 인스턴스 생성
datagen = ImageDataGenerator(rescale=1./255)

def multilabel_data_generator(image_paths, label_data, batch_size):
    num_samples = len(image_paths)
    while True:  # 무한 루프로 제너레이터 구현
        for offset in range(0, num_samples, batch_size):
            batch_images = []
            batch_labels = []

            # 배치 크기만큼 이미지와 레이블 데이터 로드 및 전처리
            batch_image_paths = image_paths[offset:offset+batch_size]
            batch_image_labels = label_data[offset:offset+batch_size]

            for img_path, labels in zip(batch_image_paths, batch_image_labels):
                img = load_img(img_path, target_size=(128, 128))  # 이미지 로드 및 크기 조정
                img = img_to_array(img)  # 이미지를 numpy 배열로 변환
                img = datagen.standardize(img)  # 데이터 전처리
                
                batch_images.append(img)
                batch_labels.append(labels)
            
            # 배치 데이터 반환
            yield np.array(batch_images), np.array(batch_labels)
            
batch_size = 32
train_generator = multilabel_data_generator(train_image_paths, train_label_data, batch_size)
validation_generator = multilabel_data_generator(valid_image_paths, valid_label_data, batch_size)
test_generator = multilabel_data_generator(test_image_paths, test_label_data, batch_size)

In [3]:
def build_model(hp):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    
    for layer in base_model.layers:
        base_model.trainable = False
    for layer in base_model.layers[-9:]:
        base_model.trainable = True
        
    # 모델 구성    
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(units=hp.Int('units', min_value=128, max_value=1024, step=128), activation = 'relu'))
    model.add(layers.Dense(5, activation='sigmoid')) # 멀티라벨 분류를 위한 sigmoid 활성화 함수 사용
        
    model.compile(optimizer=optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-3, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


In [4]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # 시도할 하이퍼파라미터 조합의 최대 개수
    executions_per_trial=1,  # 각 하이퍼파라미터 설정을 평가하기 위해 모델을 훈련시킬 횟수
    directory=r'E:\AImodel\ResNet_MultiModel',  # 튜닝 세션의 결과를 저장할 디렉토리 이름
    project_name='KneePushup_multilabel_01', # 프로젝트 이름    
)
tuner.search_space_summary()


Reloading Tuner from E:\AImodel\ResNet_MultiModel\KneePushup_multilabel_01\tuner0.json
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 128, 'sampling': 'linear'}
learning_rate (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.001, 'step': None, 'sampling': 'log'}


In [5]:
earlystopping = EarlyStopping(monitor='val_loss', patience=7, mode='min', verbose=1)

tuner.search(train_generator,
             steps_per_epoch=len(train_image_paths) // batch_size,
             epochs=25,
             validation_data=validation_generator,
             validation_steps=len(valid_image_paths) // batch_size,
             callbacks=[earlystopping])


# 최적의 하이퍼파라미터 가져오기
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# 최적의 하이퍼파라미터로 모델 빌드
model = tuner.hypermodel.build(best_hps)



Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
1024              |384               |units
1.2822e-05        |0.00072136        |learning_rate

Epoch 1/25
818/973 [========================>.....] - ETA: 16s - loss: 0.7494 - accuracy: 0.6051

KeyboardInterrupt: 

In [ ]:
def precision_m(y_true, y_pred):
    threshold = 0.5
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    predicted_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def recall_m(y_true, y_pred):
    threshold = 0.5
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    possible_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+tf.keras.backend.epsilon()))


In [ ]:
# 모든 최적 하이퍼파라미터 출력
for hp in best_hps.values:
    print(f"The optimal value for {hp} is {best_hps.get(hp)}")


model.compile(optimizer=optimizers.Adam(learning_rate=best_hps.get('learning_rate')),
              loss='binary_crossentropy',
              metrics=['accuracy', precision_m, recall_m, f1_m])

# 모델 훈련
history = model.fit(train_generator,
                    steps_per_epoch=len(train_image_paths) // batch_size,
                    epochs=25,
                    validation_data=validation_generator,
                    validation_steps=len(valid_image_paths) // batch_size,
                    callbacks=[earlystopping])
    # train_image_resized, 
    #                 train_label_data, 
    #                 validation_data=(valid_image_resized, valid_label_data), 
    #                 epochs=25, 
    #                 batch_size=32,
    #                 callbacks=[earlystopping]
                   

# 모델 평가
test_loss, test_acc, test_precision, test_recall, test_f1 = model.evaluate(test_image_resized, test_label_data)
print('\n테스트 정확도:', test_acc)
print('테스트 Precision:', test_precision)
print('테스트 Recall:', test_recall)
print('테스트 F1 Score:', test_f1)


In [ ]:
model.compile(optimizer=optimizers.Adam(learning_rate=best_hps.get('learning_rate')),
              loss='binary_crossentropy')

model.save(r'E:\AImodel\models\Face-ResNet-KneePushup-multiLabel-model')